In [ ]:
from azureml.core import Workspace, Experiment

#workspace and resources management
ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
#name of the compute cluster
cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already (good practice extracted by Microsoft official documentation: https://docs.microsoft.com/)
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
#creating compute (cpu) cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive import GridParameterSampling
from azureml.train.hyperdrive import choice
import os


# Specify parameter sampler
#define parameter search space -->random sampling over a hyperparameter search space (this would be good for first approach)
# dictionary{key:parameter, value: distribution}
#decided to go forward Grid, for more exhaustive exploration even if the runing time is much  bigger. 
ps =  GridParameterSampling({'--regularization': choice(0.001, 0.005, 0.01, 0.05, 0.1, 1.0)
    'penalty' : ['l1', 'l2'],
    'C' : np.logspace(-4, 4, 20),
    'solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']})

# Specify a Policy
# early termination for bad performance runs
# Bandit terminates runs where the primary metric is not within the specified slack factor/slack amount compared to the best performing run. (from official documentation)
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5) ### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
### YOUR CODE HERE ###

#
est = SKLearn(source_directory='.',#folder for experiment configuration files.
              entry_script=  'train.py', #relative path to the file used to start training (training script)
              #script_params = {'--C'': value,--max_iter:value }, 
              compute_target=cpu_cluster)
                     

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                hyperparameter_sampling=ps,
                                policy=policy)
                                primary_metric_name='AUC_weighted',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE)
                                #max_total_runs=4,
                                #max_concurrent_runs=4)### YOUR CODE HERE ###

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
### YOUR CODE HERE ###
#submiting the exp
hyperdrive_run = exp.submit(hd_config)
#visualizing `progress of runs
RunDetails(hyperdrive_run).show()


In [ ]:
import joblib
# Get your best run and save the model from that run.
### YOUR CODE HERE ###
#finding the best model
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print('\n learning rate:',parameter_values[3])
print('\n keep probability:',parameter_values[5])
print('\n batch size:',parameter_values[7])

#saving the best model--> find the best one 
model = best_run.register_model(model_name='best_model_lr', model_path='best_model_lr.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
#creating a tabularDataset
ds =TabularDatasetFactory.from_delimited_files('https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

#concatenate features and target
data = pd.concat([x, y])
#creating dir 'data'
if not os.path.isdir('data'):
    os.mkdir('data')
    
#saving the training dataframe to a csv for further conversion into Tabulardataset (there is not direct conversion from dataframe to Tabular or registration of a dataframe )
pd.DataFrame(data).to_csv("data/train_data.csv", index=False)
#get datastore to upload the csv
ds = ws.get_default_datastore()
#upload the dataset
datastore.upload(src_dir='data', target_path='data')
#creating a Tabulardasat from datastore
ds = TabularDatasetFactory.from_delimited_files(datastore.path('data/train_data.csv'))


In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='AUC_weighted',
    training_data=ds, #note: this should contain both features and label/target
    label_column_name='y',
    debug_log='automated_ml_errors.log',
    "feauturization": 'auto'
    "verbosity": logging.INFO)
    #"n_cross_validations": 5)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(automl_config,show_output=True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, fitted_model = automl_run.get_output()
model = best_run.register_model(model_name='best_automodel', model_path='best_automodel.joblib')